# Implementierung von Diffusion Posterior Sampling (DPS)

Hier werden wir selbst die Methode "Diffusion Posterior Sampling" (DPS) von [Chung et al., 2022](https://arxiv.org/abs/2209.14687) implementieren.
DPS ist eine Methode zum Lösen linearer und nichtlinearer inverser Probleme unter Nutzung eines vortrainierten Diffusion-Models als Prior.

Wir nutzen an einigen Stellen das [DeepInverse (`deepinv`) Python-Paket](https://deepinv.github.io/deepinv/), welches für das Lösen inverser Probleme designed wurde, damit wir nicht jede einzelne Operation von null auf selbst implementieren müssen.

## Problem-Setup
Wir definieren uns hier ein 'random inpainting' problem, bei dem zufällig (aber fix) gewählte Pixel im Bild fehlen und aus den verbleibenden Pixeln rekonstruiert werden müssen.

Wir laden dazu als erstes ein Beispielbild von Größe 64 x 64, welches wir als unsere "ground truth" (wahres Bild) nutzen. Wir repräsentieren das Bild in diesem Kontext immer als ein Array von Gleitkommazahlen (float32), dessen Werte zwischen -1.0 und 1.0 liegen.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt, matplotlib as mpl
import ipywidgets as widgets
%matplotlib widget

import deepinv as dinv
from deepinv.utils.plotting import plot
from deepinv.optim.data_fidelity import L2
from deepinv.utils.demo import load_image
from tqdm import tqdm  # to visualize progress

def todo():
    raise NotImplementedError("In dieser Zelle gibt es noch mindestens ein TODO!")
def check_for_nan(var, var_name):
    if var.isnan().any():
        raise ValueError(f"NaN (not-a-number) Wert wurde in der Variable {var_name} gefunden! Abbruch...")

device = dinv.utils.get_freer_gpu() if torch.cuda.is_available() else "cpu"

x_true = load_image("example_images/butterfly.png", img_size=64).to(device) * 2 - 1  # map from deepinv's loading range [0, 1] to [-1, 1]
print("Type of x_true:", x_true.dtype, "; min =", x_true.min(), "; max =", x_true.max())
x = x_true.clone()

Dann definieren wir uns das 'random inpainting' Problem und den zugehörigen Vorwärts-Operator, der hier `physics` genannt wird.
Wir simulieren hier, dass 90% der Pixel im Bild fehlen, und aus den verbleibenden 10% der Pixel rekonstruiert werden müssen.
Außerdem verrauschen wir die verbleibenden Pixel mit Gaussian Noise von Varianz $\sigma^2 = 0.01^2$, was die Schwierigkeit des Problems nochmal etwas erhöht.

In [ ]:
sigma = 0.01  # noise level

physics = dinv.physics.Inpainting(
    img_size=(3, x.shape[-2], x.shape[-1]),
    mask=0.1,  # wie viele Pixel (1.0 = 100%, 0.1 = 10%) sind in der Messung y übrig?
    pixelwise=True,
    noise_model=dinv.physics.GaussianNoise(sigma=sigma),
    device=device,
)

y = physics(x_true)

imgs = [y, x_true]
plot(
    imgs,
    titles=["measurement", "ground-truth"],
)

## Laden eines vortrainierten Diffusion Models

Da wir nicht genügend Ressourcen und Zeit haben, hier unser eigenes echtes Diffusion-Model auf Bildern zu trainieren, nutzen wir ein vortrainiertes Model (deep neural net / DNN).

Dieses Model wurde auf dem [FFHQ dataset](https://github.com/NVlabs/ffhq-dataset) in Auflösung 256x256 trainiert, welches nur Bilder von menschlichen Gesichtern enthält. Das ist zwar kein besonders guter Prior für unser Beispielbild von einem Schmetterlingsflügel, funktioniert aber für das gewählte inverse Problem trotzdem visuell ganz gut. Der Grund für die Nutzung des Modells ist hier hauptsächlich, dass es kein so riesiges DNN ist und daher nicht so lange braucht um ausgewertet zu werden.

In [ ]:
dnn = dinv.models.DiffUNet(large_model=False).to(device)

## Definieren der Diffusion-Schedule

Wir nutzen hier die Standard-Schedule aus [DDPM (Ho et al., 2020)](https://arxiv.org/pdf/2006.11239). Dort wird zunächst $\beta_t$ als lineare schedule zwischen gewählten Konstanten $\beta_{\rm min}$ und $\beta_{\rm max}$ definiert. Dann werden die daraus folgenden Koeffizienten definiert:

$$
\begin{align*}
\alpha_t := 1 - \beta_t\\
\bar\alpha_t := \prod_{j=1}^t \alpha_j
\end{align*}
$$

Hier nochmal wiederholt die Formeln für den Vorwärts-Diffusionsprozess vom Freitag:

$$
\begin{align*}
\mathbf{x}_t | \mathbf{x}_{t-1}, \varepsilon &= \sqrt{\alpha_t}\mathbf{x}_{t-1} + \sqrt{1 - \alpha_t}\varepsilon, \qquad \varepsilon \sim \mathcal{N}(0, I)  \qquad (\text{schrittweise})\\
\mathbf{x}_t | \mathbf x_0, \varepsilon' &= \sqrt{\bar\alpha_t}\mathbf{x}_0 + \sqrt{1 - \bar\alpha_t}\mathbf \varepsilon',\qquad \varepsilon' \sim \mathcal{N}(0, I) \qquad (\text{direkt von}\ \mathbf x_0)
\end{align*}
$$

Trainiert wurde das DNN mit 1000 Schritten, was ihr auch an `betas.shape` sehen könnt.

In [ ]:
# These are the parameters that were used for the noise schedule during model training

num_train_timesteps = 1000  # Number of timesteps used during training
beta_min = 0.1 / 1000
beta_max = 20 / 1000
betas = torch.linspace(beta_min, beta_max, num_train_timesteps).to(device)
alphas = (1 - betas)
alpha_bars = alphas.cumprod(dim=0)

betas.shape

# Definition eines Denoisers basierend auf dem Diffusion-Model

Da das Diffusion-Model effektiv dafür trainiert wurde, den Noise-Anteil $z$ in einem Bild $\mathbf x_t = \sqrt{\bar{\alpha}_t} \mathbf x_0 + \sqrt{1 - \bar{\alpha}_t} \mathbf z$ zu schätzen, können wir es auch als Denoiser von $\mathbf x_t$ einsetzen. Gegeben $\mathbf x_t$ und $t$ gibt uns das Diffusion-Model $\hat{\varepsilon} = D_\theta(\mathbf x_t, t)$ als eine Schätzung von $\varepsilon$. Wir können die Gleichung einfach umstellen und bekommen:

$$\hat{\mathbf x}_0 = \frac{\mathbf x_t - (1 - \sqrt{\bar{\alpha}_t}) \hat{\mathbf \varepsilon}}{\sqrt{\bar{\alpha}_t}}$$


**Aufgabe:**
* Implementiert $\hat{\mathbf x}_0$ in der `denoiser(xt, t)` Funktion. Nutzt die dort vordefinierte Variable `noise_estimate`, die das geschätzte $\hat{\varepsilon}$ enthält.

In [ ]:
def get_dnn_noise_estimate(xt, t):
    """A little wrapper around the DNN forward() call to return only the estimated noise"""
    noise_est_sample_var = dnn(xt, t, type_t="timestep")
    noise_est = noise_est_sample_var[:, :3, ...]
    return noise_est


def denoiser(xt, t):
    if isinstance(t, int):
        # convert integer timesteps to timestep tensors
        t = torch.tensor([t]*xt.shape[0], device=xt.device, dtype=torch.long)
    # this is an estimate of the zero-mean unit-variance added Gaussian noise sample epsilon ~ N(0, I)
    epshat = get_dnn_noise_estimate(xt, t)

    ### PARTICIPANT TODO

    # x0_hat = todo()

    ### PARTICIPANT TODO END

    return x0_hat, epshat

In [ ]:
t = 200  # as an example for single-step denoising: choose some arbitrary timestep in [0, 999]

# Draw x_t sample based on ground-truth x_0
x0 = x_true
xt = alpha_bars[t].sqrt() * x0 + (1 - alpha_bars[t]).sqrt() * torch.randn_like(x0)

# Apply the denoiser 
x0_t, epshat_t = denoiser(xt, t)

# Visualize
imgs = [x0, xt, x0_t, epshat_t]
plot(
    imgs,
    titles=["ground-truth", "noisy", r"denoised $\widehat{\mathbf x}_0$", r"noise estimate $\hat{\varepsilon}$"],
)

# Der DPS-Algorithmus

Nachdem wir jetzt ein Vorwärts-Modell `physics` und einen diffusion-basierten Denoiser `denoiser(xt, t)` definiert haben, können wir den DPS-Algorithmus implementieren.

DPS kombiniert in einem Bayes-inspirierten Sinne einen Gradienten bezüglich der *likelihood*, gegeben durch das Vorwärts-Modell, mit einem Gradienten bezüglich des *priors*, gegeben durch das Diffusion Model.

## Wiederholung von der Tafel:

Um gradienten-basiertes **posterior sampling** mit einem Diffusion-Model zu realisieren, möchten wir den folgenden Term ausrechnen:
$$\nabla_{\mathbf{x}_t} \log p(\mathbf{x}_t|\mathbf{y})$$

Durch den Satz von Bayes, die Eigenschaften des Logarithmus, und des Gradienten gelangen wir zu:

\begin{align*}\nabla_{\mathbf{x}_t} \log p(\mathbf{x}_t|\mathbf{y}) = \nabla_{\mathbf{x}_t} \log p(\mathbf{x}_t)
          + \nabla_{\mathbf{x}_t} \log p(\mathbf{y}|\mathbf{x}_t)\end{align*}

**Frage:** Wo ist der Term $p(y)$ aus dem Satz von Bayes hin...?

Für den ersten der beiden Terme können wir unser Vorwärts-Modell nutzen, indem wir durch dieses durch differenzieren. Die Aufgabe übernimmt hier `deepinv` bzw. PyTorch für uns.
Der zweite Term steht uns auch nach dem Training eines Diffusion-Models leider nicht zur Verfügung, aus theoretischen Gründen – welche wir, wenn ihr das hier lest, bereits besprochen haben sollten :).
DPS schlägt zur Approximation dieses Terms Folgendes vor (siehe Theorem 1 aus [Chung et al., 2022](https://arxiv.org/abs/2209.14687) für Details):

\begin{align*}\nabla_{\mathbf{x}_t} \log p(\mathbf{x}_t|\mathbf{y}) \approx \nabla_{\mathbf{x}_t} \log p(\mathbf{x}_t)
          + \nabla_{\mathbf{x}_t} \log p(\mathbf{y}|\widehat{\mathbf{x}}_{0}(\mathbf{x_t}, t))\end{align*}

wobei wir zum Ausrechnen von $\widehat{\mathbf x}_0(\mathbf x_t, t)$ genau unseren oben definierten Denoiser nutzen können! :)
Unter der Modell-Annahme, dass $y$ Gauss-verteilt ist (siehe unser noise model oben), gilt für die log-probability im zweiten Term:



\begin{align*}\log p(\mathbf{y}|\widehat{\mathbf{x}}_0(\mathbf{x_t})) =
      -\frac{\|\mathbf{y} - A\widehat{\mathbf{x}}_0(\mathbf{x_t}, t)\|_2^2}{2\sigma_y^2}.\end{align*}

und um den zugehörigen Gradienten zu bekommen, nutzen wir die *automatic differentiation* Fähigkeiten von PyTorch (`torch.autograd`).

**Aufgabe**: In der nächsten Zelle testen wir das für ein einzelnes $t = 150$. Führt die nächste Zelle aus und versucht den kommentierten Abschnitt "DPS likelihood gradient" nachzuvollziehen.

In [ ]:
x0 = x_true

get_likelihood_loss = L2()  # L2-Loss für den likelihood loss

# Sample xt ~ q(xt|x0)
t = 150  # choose some arbitrary timestep
at = alpha_bars[t]
sigma_cur = (1 - at).sqrt()
xt = at.sqrt()*x0 + sigma_cur * torch.randn_like(x0)

# DPS likelihood gradient
with torch.enable_grad():
    # Turn on gradients for xt so we can use autograd
    xt.requires_grad_()
    # Call the denoiser, get estimates for clean image x0_t and noise epshat_t
    x0_t, epshat_t = denoiser(xt, t)
    # Log-likelihood
    likelihood_loss = get_likelihood_loss(x0_t, y, physics).sum()
    # Take gradient w.r.t. xt
    likelihood_grad = torch.autograd.grad(outputs=likelihood_loss, inputs=xt)[0]

# Visualize
imgs = [x0, xt, x0_t, epshat_t, likelihood_grad]
plot(
    imgs,
    titles=["ground-truth", f"noisy ({t=})", r"denoised $\hat{\mathbf x}_0$", r"noise estimate $\hat{\varepsilon}$", "likelihood gradient"],
)

Hier ein Screenshot von Pseudocode des Algorithmus aus dem [DPS-Paper](https://arxiv.org/pdf/2209.14687), siehe dort "Algorithm 1":

![image.png](dps_pseudocode.png)

**Aufgabe**: Implementiert den DPS-Algorithmus in Python.
<br>
**Hinweise:**
* Der Index $i$ meint hier immer unsere Diffusion Time $t$, und wir nutzen im Code immer $t$ als Index.
* $\mathbf z$ meint hier unser $\varepsilon$, also das noise sample.
* Ein paar Größen sind für euch vordefiniert, damit ihr nicht alles nachschlagen müsst, insbesondere $\tilde{\sigma}_i$ (`sigma_t`) und $\nabla_{x_i} \lVert \mathbf y - \mathcal{A}(\hat{\mathbf x}_0) \rVert_2$ (`likelihood_grad`), sowie $\alpha_i, \bar{\alpha}_i$, $\bar{\alpha}_{i-1}$, $\zeta_i$ (`a_t`, `abar_t`, `abar_t_next`, `zeta_t`).
* Da ein Durchlauf mit 1000 Iterationen leider 30 Sekunden bis zu ein paar Minuten läuft, ist es zu empfehlen, eure Lösung etwas genauer anzuschauen, bevor ihr sie nach jeder Änderung sofort ausprobiert (:
<!-- * Implementiert `zeta_t` = $\zeta_i := \zeta / \big\lVert y - A(\widehat{\mathbf x}_0) \big\rVert_2$, wobei `zeta` = $\zeta$ ein frei gewählter konstanter Hyperparameter ist.
    * Hier ist `torch.linalg.vector_norm` mit `dim=(-2, -1)` eine hilfreiche Funktion.
    * Wir haben schon mal `zeta = 0.5` vordefiniert, womit unser inverses Problem hier ganz gut klappen sollte, wenn eure Implementation stimmt.
-->

In [ ]:
# Algorithm hyperparameters:
# * zeta: the likelihood gradient scale/strength. usually the sweet spot is somewhere between 0.1 and 3.0
zeta = 1.0

# simulate measurement
x0 = x_true
y = physics(x0.to(device))

# sample an initial image estimate: just a sample from x_T, where p(x_T) = N(0, I)
x = torch.randn_like(x0)
xs = [x]
x0_preds = []

with torch.no_grad():
    for t in tqdm(reversed(range(0, num_train_timesteps)), total=num_train_timesteps):
        a_t = alphas[t]
        abar_t = alpha_bars[t]
        abar_t_next = alpha_bars[t - 1] if t - 1 >= 0 else torch.tensor(1)
        b_t = betas[t]
        
        sigma_tilde = torch.sqrt(b_t * (1 - abar_t_next) / (1 - abar_t))  # see Eq. 7 in DDPM paper

        # Get the denoised image x0_hat and its likelihood gradient
        xt = xs[-1].to(device)
        with torch.enable_grad():
            xt.requires_grad_()
            # 1. denoising step
            x0_hat, eps_hat = denoiser(xt, t)
            # 2. likelihood gradient approximation
            likelihood_loss = get_likelihood_loss(x0_hat, y, physics).sum()
        likelihood_grad = torch.autograd.grad(outputs=likelihood_loss, inputs=xt)[0].detach()
        zeta_t = zeta / torch.linalg.vector_norm(likelihood_grad, ord=2, dim=(-2, -1), keepdim=True)
        eps = torch.randn_like(xt)  # new noise sample for this iteration, different from eps_hat

        ### PARTICIPANT TODO

        # xt_next_prime = todo()
        # zeta_t = todo()
        # xt_next = xt_next_prime - todo()

        ### PARTICIPANT TODO END

        check_for_nan(x0_hat, "x0_hat")
        check_for_nan(xt_next, "xt_next")
        x0_preds.append(x0_hat.to("cpu"))
        xs.append(xt_next.to("cpu"))

recon = xs[-1]

# plot the results
imgs = [y, recon, x_true]
plot(imgs, titles=["measurement", "model output", "ground-truth"])

In [ ]:
num_steps = num_train_timesteps

fig, axs = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(7, 2.5))
shown_measurement = y.detach().cpu()[0].moveaxis(0, 2).clip(min=-1, max=1)
shown_measurement[shown_measurement.isclose(torch.tensor(0.0))] = np.nan  # hide masked-out pixels (zeros)
im0 = axs[0].imshow((shown_measurement + 1) / 2)
im1 = axs[1].imshow((xs[-1].detach().cpu()[0].moveaxis(0, 2).clip(min=-1, max=1) + 1) / 2)
im2 = axs[2].imshow((x_true.detach().cpu()[0].moveaxis(0, 2).clip(min=-1, max=1) + 1) / 2)
axs[0].set_title('Measurement')
axs[1].set_title('Model output')
axs[2].set_title('Ground truth')
for ax in axs:
    ax.axis('off')
fig.tight_layout()

@widgets.interact(t=(0, num_steps-1, 1))
def update(t=0):
    im1.set_data((xs[num_steps - t - 1].detach().cpu()[0].moveaxis(0, 2).clip(min=-1, max=1) + 1) / 2)

# Bonus 1: Mit anderen Bildern rumspielen

Ladet ganz oben statt `example_images/butterfly.png` andere Bilder aus dem Ordner `example_images`. Setzt dabei noch die Option `resize_mode='resize'` in der `load_image` Funktion. Falls ihr eine GPU habt, könnt ihr auch versuchen den `img_size` Parameter wegzunehmen, sodass die volle Auflösung (256 x 256) genutzt wird. Probiert dann z.B. verschiedene `zeta` Einstellungen.

---

# Bonus 2: Eine einfachere Formulierung mit der Möglichkeit, Zeitschritte zu überspringen

(Ab hier nur Bonus für Leute die noch Zeit und Lust haben!)

Es ist ein bisschen sehr kostspielig und langwierig, das neuronale Netz 1000 mal auszuwerten. Wir können hier aber durch ein paar geschickte Ideen Zeitschritte überspringen und so nur bspw. 200 Auswertungen erreichen.

Zeile 6 des DPS-Algorithmus definiert $\mathbf x_{t-1}'$ auf Basis der Größen
$$\{ \mathbf x_t, \hat{\mathbf x}_0, \mathbf \varepsilon_{\text{next}} \}$$
wobei wir hier $\mathbf \varepsilon_{\text{next}}$ für den neu gesampelten Gaussian Noise schreiben, um ihn klarer von dem aus dem vorherigen Bild geschätzten Noise zu unterscheiden. Da $\mathbf x_t$ aber einfach eine gewichtete Summe aus $\hat{\mathbf x}_0$ und der Schätzung des vorherigen Noise $\hat{\mathbf \varepsilon}$ ist, kann man $\mathbf x_{t-1}'$ aber stattdessen auch auf Basis der Größen
$$\{ \hat{\mathbf x}_0, \hat{\mathbf \varepsilon}, \mathbf \varepsilon_{\text{next}} \}$$ schreiben.

**Aufgabe**: Leitet euch durch Umstellen den expliziten Ausdruck für $\mathbf x_t$ als Funktion von $\hat{\mathbf x}_0$ und $\hat{\mathbf \varepsilon}$ her und schreibt ihn euch auf.

Man kann dann algebraisch mit einigem Aufwand zeigen, dass Zeile 6 aus DPS äquivalent ist zu:
$$
\mathbf x_{t-1}' = \sqrt{\bar{\alpha}_{t-1}} \hat{\mathbf x}_0 + \sqrt{1 - \bar{\alpha}_{t-1} - \tilde{\sigma}_t^2} \hat{\mathbf \varepsilon} + \tilde{\sigma}_t \mathbf \varepsilon_{\text{next}}
$$

**Frage**: Was sind Vorteile dieses Ausdrucks gegenüber Zeile 6 aus DPS, wenn man darüber nachdenkt, Zeitschritte zu überspringen?

**Antwort**:
Der Ausdruck hängt nicht mehr explizit von Konstanten zum aktuellen Zeitschritt $t$ ab, sondern nur von Konstanten zum nächsten Zeitschritt $t-1$. Wir können damit nun auch Zeitschritte überspringen, indem wir statt $t-1$ immer $t-s$ schreiben, für einen "Skip"-Parameter $s$. Das erlaubt uns schnelleres Sampling, da wir die Schleife bspw. nur 200 statt 1000 mal durchlaufen müssen.

**Aufgabe**:
* Implementiert den neuen Ausdruck für `xt_next_prime` im Code hierunter.
* Kopiert euren Ausdruck für `xt_next` auf Basis von `xt_next_prime` von eurer ersten DPS-Implementierung.
* Führt dann die Zelle aus, probiert verschiedene `num_steps` und `zeta` aus und diskutiert eure Beobachtungen.

**Fragen**:
* Warum können wir hier nicht ein völlig beliebiges `num_steps` setzen, z.B. 172?
* Warum müssen wir evtl. `zeta` anpassen, wenn wir `num_steps` ändern?

In [ ]:
# Algorithm hyperparameters:
# * zeta: the likelihood gradient scale/strength. usually the sweet spot is somewhere between 0.1 and 3.0
# * num_steps: number of steps, used for skipping
zeta = 1.0  # set to a reasonable default for num_steps = 100
num_steps = 200
assert num_steps <= 1000, "more than 1000 steps not possible with this model"
skip = num_train_timesteps // num_steps  # how many steps are we skipping each iteration due to num_steps < 1000?

# simulate measurement
x0 = x_true
y = physics(x0.to(device))

# sample an initial image estimate: just a sample from x_T, where p(x_T) = N(0, I)
x = torch.randn_like(x0)
xs_fast = [x]
x0_preds_fast = []

with torch.no_grad():
    # loop step skipping happens here (range with step size "skip")
    for t in tqdm(reversed(range(0, num_train_timesteps, skip)), total=num_steps):
        a_t = alphas[t]
        b_t = betas[t]
        abar_t = alpha_bars[t]
        abar_t_next = alpha_bars[t - skip] if t - skip >= 0 else torch.tensor(1)

        xt = xs_fast[-1].to(device)
        with torch.enable_grad():
            xt.requires_grad_()
            # 1. denoising step
            x0_hat, eps_hat = denoiser(xt, t)
            # 2. likelihood gradient approximation
            likelihood_loss = get_likelihood_loss(x0_hat, y, physics).sum()
        likelihood_grad = torch.autograd.grad(outputs=likelihood_loss, inputs=xt)[0].detach()
        zeta_t = zeta / torch.linalg.vector_norm(likelihood_grad, ord=2, dim=(-2, -1), keepdim=True).detach()
        
        # 3. Denoising / Renoising step, determines x_{t-skip}' aka xt_next_prime
        #  -> See Algorithm 1 (DPS - Gaussian) in DPS (Chung et al. 2022) paper
        eps_next = torch.randn_like(xt)
        sigma_tilde = torch.sqrt(b_t * (1 - abar_t_next) / (1 - abar_t))  # see Eq. 7 in DDPM paper

        # reformulation in terms of x0hat, noise_estimate and z       
        xt_next_prime = abar_t_next.sqrt() * x0_hat + (1 - abar_t_next - sigma_tilde**2).sqrt() * eps_hat \
            + sigma_tilde * eps_next

        # 4. Measurement likelihood step, determines x_{t-skip} aka xt_next
        #  -> We apply the step size scaling proposed in DPS,
        #     see footnote 5 on page 6 of the DPS paper,
        #     and then apply the gradient with the calculated step size.
        xt_next = xt_next_prime - zeta_t * likelihood_grad

        x0_preds_fast.append(x0_hat.to("cpu"))
        xs_fast.append(xt_next.to("cpu"))

recon = xs_fast[-1]

# plot the results
imgs = [y, recon, x_true]
plot(imgs, titles=["measurement", "model output", "ground-truth"])

In [ ]:
fig_f, axs_f = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(7, 2.5))
shown_measurement = y.detach().cpu()[0].moveaxis(0, 2).clip(min=-1, max=1)
shown_measurement[shown_measurement.isclose(torch.tensor(0.0))] = np.nan  # hide masked-out pixels (zeros)
im0_f = axs_f[0].imshow((shown_measurement + 1) / 2)
im1_f = axs_f[1].imshow((xs_fast[-1].detach().cpu()[0].moveaxis(0, 2).clip(min=-1, max=1) + 1) / 2)
im2_f = axs_f[2].imshow((x_true.detach().cpu()[0].moveaxis(0, 2).clip(min=-1, max=1) + 1) / 2)
axs_f[0].set_title('Measurement')
axs_f[1].set_title('Model output')
axs_f[2].set_title('Ground truth')
for ax in axs_f:
    ax.axis('off')
fig_f.tight_layout()

@widgets.interact(t=(0, num_steps-1, 1))
def update(t=0):
    im1_f.set_data((xs_fast[num_steps - t - 1].detach().cpu()[0].moveaxis(0, 2).clip(min=-1, max=1) + 1) / 2)